In [ ]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [ ]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [ ]:
Campaign_Query='''



with Camp_data_TP4 as (

select distinct BACCT_NUM as BAN

from `divgpras-pr-579355.SHS.SHS_ALR_Campaign_data_SAS`
where CAMP_CREATIVE in ('ALR1_TP4')
and date(Campaign_date)<'2022-01-01' 
and OPENED=1

)

, Camp_data_TP4X as (

select distinct BACCT_NUM as BAN

from `divgpras-pr-579355.SHS.SHS_ALR_Campaign_data_SAS`
where CAMP_CREATIVE in ('ALR1_TP4X')
and date(Campaign_date)<'2022-01-01' 

)


, Creative_data as (

select distinct BACCT_NUM as BAN, 1 as creative_flag
from `divgpras-pr-579355.SHS.SHS_ALR_Campaign_data_SAS`
where REGEXP_CONTAINS (CAMP_CREATIVE, r'ALR1_TP5')
-- and  date(Campaign_date)>='2022-01-01' and date(Campaign_date)<='2022-06-30'

)


,merged_data as (

select * from Camp_data_TP4

UNION ALL

select * from Camp_data_TP4X

)



, Final_data as (

select distinct a.BAN, b.creative_flag 
from merged_data a
left join Creative_data b
on a.BAN=b.BAN

)
select distinct BAN from  Final_data
where creative_flag is Null


'''

In [ ]:
Camp_DF=extract_bq_data(bq_client, sql=Campaign_Query)

In [ ]:
Camp_DF.info()

In [ ]:
# pd.DataFrame(Camp_DF.groupby(['CAMP_CREATIVE','OPENED']).agg(
    
#     Customer_count= ('BACCT_NUM','nunique'),
#     # Email_Opened_count=('OPENED','sum'),
#     # CT_Opened_count=('CLICKTHROUGH','sum'),
#     # softbounce_count=('SOFTBOUNCE','sum'),
#     # Hardbounce_count=('HARDBOUNCE','sum'),
#     # Unsub_count=('UNSUBSCRIBE','sum'),
#     campaign_date_min=('Campaign_date','min'),
#     campaign_date_max=('Campaign_date','max')

    
#     # # Customer_Share= ('customer_id',lambda x:x.count()*100/Merge_DF_4.shape[0])
#     # Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
#     # Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)
# ).reset_index())

In [ ]:
# def Camp_creative_making(row):
    
#     if row['CAMP_CREATIVE']=='ALR1_TP5X_V1' or row['CAMP_CREATIVE']=='ALR1_TP5_V1':
#         return 'V1'
#     elif row['CAMP_CREATIVE']=='ALR1_TP5X_V2' or row['CAMP_CREATIVE']=='ALR1_TP5_V2':
#         return 'V2'
#     elif row['CAMP_CREATIVE']=='ALR1_TP5X_V3' or row['CAMP_CREATIVE']=='ALR1_TP5_V3':
#         return 'V3'
#     elif row['CAMP_CREATIVE']=='ALR1_TP5X_V4' or row['CAMP_CREATIVE']=='ALR1_TP5_V4':
#         return 'V4'
#     else:
#         return "None"

In [ ]:
# Camp_DF['Campaign_date'].value_counts().sort_index()

In [ ]:
# Camp_DF['CAMP_CREATIVE_New']=Camp_DF.apply(Camp_creative_making,axis=1)

In [ ]:
# Camp_DF['CAMP_CREATIVE_New'].value_counts().sort_index()

In [ ]:
# pd.DataFrame(Camp_DF.groupby(['CAMP_CREATIVE_New','OPENED']).agg(
    
#     Customer_count= ('BACCT_NUM','nunique'),
#     # Email_Opened_count=('OPENED','sum'),
#     # CT_Opened_count=('CLICKTHROUGH','sum'),
#     # softbounce_count=('SOFTBOUNCE','sum'),
#     # Hardbounce_count=('HARDBOUNCE','sum'),
#     # Unsub_count=('UNSUBSCRIBE','sum'),
#     campaign_date_min=('Campaign_date','min'),
#     campaign_date_max=('Campaign_date','max')

    
#     # # Customer_Share= ('customer_id',lambda x:x.count()*100/Merge_DF_4.shape[0])
#     # Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
#     # Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)
# ).reset_index())

In [ ]:
config= bigquery.job.LoadJobConfig()
config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

Table_BQ = 'SHS.SHS_ALR_Campaign_data_Control_Eligble_not_recieved'

bq_table_instance= bq_client.load_table_from_dataframe(Camp_DF, Table_BQ,job_config=config)

In [ ]:
H2_2022_Snapshot='''




DECLARE target_date_start_H2 DATE DEFAULT "2022-07-01";
DECLARE target_date_start DATE DEFAULT "2022-12-01";
DECLARE target_date_end DATE DEFAULT "2022-12-31";
DECLARE target_date_start_next_month DATE DEFAULT "2023-01-01";



With Camp_V1_BAN as (

select distinct BAN from `divgpras-pr-579355.SHS.SHS_ALR_Campaign_data_Control_Eligble_not_recieved`

)


, prod_snpsht_SMHM_start_H2 as (


select distinct bacct_bus_bacct_num as BAN,1 as count_of_BAN_FFH_start_H2
FROM `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
where pi_prod_instnc_typ_cd='SMHM' 
and pi_prod_instnc_stat_cd in ('A')
and  consldt_cust_typ_cd = 'R'
and DATE(prod_instnc_ts)=target_date_start_H2


)

, prod_snpsht_SMHM_end_H2 as (


select distinct bacct_bus_bacct_num as BAN,1 as count_of_BAN_FFH_end_H2
FROM `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
where pi_prod_instnc_typ_cd='SMHM' 
and pi_prod_instnc_stat_cd in ('A')
and  consldt_cust_typ_cd = 'R'
and DATE(prod_instnc_ts)=target_date_end


)
, ARPU_Calculation_start_H1 as(
SELECT
  distinct billg_acct_num AS ban,
  EXTRACT(YEAR
  FROM
    bill.bill_dt)||'-'||LPAD(CAST(EXTRACT(MONTH
      FROM
        bill.bill_dt) AS STRING), 2, '0') AS bill_year_month,
  sum(bill.rid_unit_chrg_amt) as ARPU_SMHM,
  sum(tot_inv_amt) as ARPU_FFH
  FROM
  `cio-datahub-enterprise-pr-183a.ent_cust_bill.bq_wln_inv_sum_view` bill
WHERE
  DATE(bill.bill_dt)>=  '2021-12-01'
  and DATE(bill.bill_dt)< '2022-01-01'
  group by ban,bill_year_month
)


, ARPU_Calculation_start_H2 as(
SELECT
  distinct billg_acct_num AS ban,
  EXTRACT(YEAR
  FROM
    bill.bill_dt)||'-'||LPAD(CAST(EXTRACT(MONTH
      FROM
        bill.bill_dt) AS STRING), 2, '0') AS bill_year_month,
  sum(bill.rid_unit_chrg_amt) as ARPU_SMHM,
  sum(tot_inv_amt) as ARPU_FFH
  FROM
  `cio-datahub-enterprise-pr-183a.ent_cust_bill.bq_wln_inv_sum_view` bill
WHERE
  DATE(bill.bill_dt)>=  '2022-06-01'
  and DATE(bill.bill_dt)< '2022-07-01'
  group by ban,bill_year_month
)

, ARPU_Calculation_end_H2 as(
SELECT
  distinct billg_acct_num AS ban,
  EXTRACT(YEAR
  FROM
    bill.bill_dt)||'-'||LPAD(CAST(EXTRACT(MONTH
      FROM
        bill.bill_dt) AS STRING), 2, '0') AS bill_year_month,
  sum(bill.rid_unit_chrg_amt) as ARPU_SMHM,
  sum(tot_inv_amt) as ARPU_FFH
  FROM
  `cio-datahub-enterprise-pr-183a.ent_cust_bill.bq_wln_inv_sum_view` bill
WHERE
  DATE(bill.bill_dt)>=  '2022-12-01'
  and DATE(bill.bill_dt)< '2023-01-01'
  group by ban,bill_year_month
)

select a.BAN
,b.count_of_BAN_FFH_start_H2
,d.count_of_BAN_FFH_end_H2
,c.bill_year_month as bill_year_month_H1_2022_start
,c.ARPU_SMHM as ARPU_SMHM_H1_2022_start
,c.ARPU_FFH as ARPU_FFH_H1_2022_start
,e.bill_year_month as bill_year_month_H2_2022_start
,e.ARPU_SMHM as ARPU_SMHM_H2_2022_start
,e.ARPU_FFH as ARPU_FFH_H2_2022_start
,f.bill_year_month as bill_year_month_H2_2022_end
,f.ARPU_SMHM as ARPU_SMHM_H2_2022_end
,f.ARPU_FFH as ARPU_FFH_H2_2022_end

 from Camp_V1_BAN a
left join prod_snpsht_SMHM_start_H2 b
on a.BAN=b.BAN
left join prod_snpsht_SMHM_end_H2 d
on a.BAN=d.BAN
left join ARPU_Calculation_start_H1 c
on a.BAN=c.ban
left join ARPU_Calculation_start_H2 e
on a.BAN=e.ban
left join ARPU_Calculation_end_H2 f
on a.BAN=f.ban


'''

In [ ]:
H2_2022_Snapshot_count=extract_bq_data(bq_client, sql=H2_2022_Snapshot)

In [ ]:
H2_2022_Snapshot_count.info()

In [ ]:
H2_2022_Snapshot_count['Place_holder']='Place_holder'

In [ ]:
H2_2022_Snapshot_count.head()

In [ ]:
pd.DataFrame(H2_2022_Snapshot_count.groupby(['Place_holder']).agg(
    
    Customer_count= ('BAN','nunique'),
    Customer_count_1= ('count_of_BAN_FFH_start_H2','sum'),
    Customer_count_2= ('count_of_BAN_FFH_end_H2','sum'),
    ARPU_SMHM_H1_2022_start=('ARPU_SMHM_H1_2022_start','mean'),
    ARPU_SMHM_H2_2022_start=('ARPU_SMHM_H2_2022_start','mean'),
    ARPU_SMHM_H2_2022_end=('ARPU_SMHM_H2_2022_end','mean'),
    ARPU_FFH_H1_2022_start=('ARPU_FFH_H1_2022_start','mean'),
    ARPU_FFH_H2_2022_start=('ARPU_FFH_H2_2022_start','mean'),
    ARPU_FFH_H2_2022_end=('ARPU_FFH_H2_2022_end','mean'),
    # Email_Opened_count=('OPENED','sum'),
    # CT_Opened_count=('CLICKTHROUGH','sum'),
    # softbounce_count=('SOFTBOUNCE','sum'),
    # Hardbounce_count=('HARDBOUNCE','sum'),
    # Unsub_count=('UNSUBSCRIBE','sum'),
    # campaign_date_min=('Campaign_date','min'),
    # campaign_date_max=('Campaign_date','max')

    
    # # Customer_Share= ('customer_id',lambda x:x.count()*100/Merge_DF_4.shape[0])
    # Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    # Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)
).reset_index())

In [ ]:
H2_2022_Snapshot_count['BAN'].value_counts()

In [ ]:
H2_2022_Snapshot_count[H2_2022_Snapshot_count['BAN']==208624864]

In [ ]:
# H2_2022_Snapshot_count.to_csv('ALR_Campaign_Control_2022H2.csv',index=False)

In [ ]:
Contract_status_Query='''






with Intial_snapshot as (
        
     SELECT 
     distinct b.BAN as ban_start_H1
     ,a.pi_prod_instnc_stat_cd as status_intial_start_H1
    , max(pi_cntrct_end_ts) as latest_contract_end_dt_start_h1
    ,pi_prod_instnc_resrc_str as res_id_intial

    FROM `divgpras-pr-579355.SHS.SHS_ALR_Campaign_data_Control_Eligble_not_recieved` b 
    LEFT JOIN `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` a
    
    ON a.bacct_bus_bacct_num = b.BAN 

    WHERE DATE(prod_instnc_ts) = '2022-01-01'
    and a.pi_prod_instnc_typ_cd='SMHM'
    and a.pi_prod_instnc_stat_cd='A'

    GROUP BY 
     b.BAN,a.pi_prod_instnc_stat_cd,a.pi_prod_instnc_resrc_str

)


, H2_start_snapshot as (
        
        SELECT
     distinct b.BAN as ban_start_h2
    ,a.pi_prod_instnc_stat_cd as status_start_h2
    , max(pi_cntrct_end_ts) as contract_end_dt_start_h2
     ,pi_prod_instnc_resrc_str as res_id_start_h2
    FROM `divgpras-pr-579355.SHS.SHS_ALR_Campaign_data_Control_Eligble_not_recieved` b 
    LEFT JOIN`cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` a
    
    ON a.bacct_bus_bacct_num = b.BAN 

    WHERE DATE(prod_instnc_ts) = '2022-07-01'
    and a.pi_prod_instnc_typ_cd='SMHM'

    GROUP BY 
      b.BAN,a.pi_prod_instnc_stat_cd,a.pi_prod_instnc_resrc_str

)

, H2_end_snapshot as (
        
        SELECT
     distinct b.BAN as ban_end_h2
    ,a.pi_prod_instnc_stat_cd as status_end_h2
    , max(pi_cntrct_end_ts) as contract_end_dt_end_h2
    ,pi_prod_instnc_resrc_str as res_id_end_h2

    FROM `divgpras-pr-579355.SHS.SHS_ALR_Campaign_data_Control_Eligble_not_recieved` b 
    LEFT JOIN`cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` a
    
    ON a.bacct_bus_bacct_num = b.BAN 

    WHERE DATE(prod_instnc_ts) = '2022-12-31'
    and a.pi_prod_instnc_typ_cd='SMHM'

    GROUP BY 
     b.BAN,a.pi_prod_instnc_stat_cd,a.pi_prod_instnc_resrc_str

)




select 
distinct a.ban_start_H1,a.status_intial_start_H1,a.latest_contract_end_dt_start_h1,b.*,c.*
from Intial_snapshot a
left JOIN  H2_start_snapshot b
on a.ban_start_H1=b.ban_start_h2 and a.res_id_intial=b.res_id_start_h2
left join H2_end_snapshot c
on a.ban_start_H1=c.ban_end_h2 and a.res_id_intial=c.res_id_end_h2

'''

In [ ]:
Contract_DF=extract_bq_data(bq_client, sql=Contract_status_Query)

In [ ]:
Contract_DF.info()

In [ ]:
Contract_DF.head()

In [ ]:
Contract_DF['ban_start_H1'].value_counts()

In [ ]:
Contract_DF[Contract_DF.ban_start_H1==601929358]

In [ ]:
Contract_DF.to_csv('ALR_Campaign_Control_eligible_not_received_status_2022H2.csv',index=False)